In [ ]:
import os
import pandas as pd
from huggingface_hub import HfApi, login
login(
    add_to_git_credential=True
)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (manager).
Your token has been saved to C:\Users\MSI Laptop\.cache\huggingface\token
Login successful


In [24]:
model_results_and_actual_data_folder_name = os.path.join("..", "5) BERT Based Models", "Model Results and Actual Data")
model_evaluation_results_folders = {
    "BERT": ["Hyperparameter-1", "Hyperparameter-2", "Hyperparameter-3", "Hyperparameter-4"],
    "DistilBERT": ["Hyperparameter-1", "Hyperparameter-2", "Hyperparameter-3", "Hyperparameter-4"]
}
model_repo_name = "u-kuro/Thesis-2024-U.S.-Presidential-Election-Sentiment-Analysis"

In [25]:
# Define Function to Retrieve Saved Model
def get_best_accuracy_model():
    # Check best model accuracy
    best_model_folder_path = None
    best_model_accuracy = None
    best_model_name = None
    
    for model_folder in model_evaluation_results_folders:

        model_parameters = []
        
        for hyperparameter_folder in model_evaluation_results_folders[model_folder]:
            folder_path = os.path.join(
                model_results_and_actual_data_folder_name,
                model_folder,
                hyperparameter_folder
            )
            
            # Find prediction files
            unique_file_suffix = " - Test Metric.csv"
            prediction_files = [f for f in os.listdir(folder_path) if f.endswith(unique_file_suffix)]
            
            if len(prediction_files) == 0:
                raise Exception(f'Missing File with "{unique_file_suffix}" suffix in "{folder_path}"')
            elif len(prediction_files) > 1:
                raise Exception(f'File with "{unique_file_suffix}" suffix in "{folder_path}" is duplicated')
                
            # Get the prediction file
            file = prediction_files[0]
            file_path = os.path.join(folder_path, file)
                
            # Load data
            df = pd.read_csv(file_path)
            test_metrics = df.iloc[0].to_dict()

            # Get accuracy
            accuracy = test_metrics['accuracy']
            if accuracy is None:
                raise Exception(f'Missing Test Metric "accuracy" in "{file_path}"')
            
            if best_model_accuracy is None or accuracy > best_model_accuracy:
                best_model_accuracy = accuracy
                best_model_folder_path = folder_path
                best_model_name = model_folder

    # Get saved best model
    unique_folder_suffix = " - Model"
    model_folder = [f for f in os.listdir(best_model_folder_path) if f.endswith(unique_folder_suffix)]
    
    if len(model_folder) == 0:
        raise Exception(f'Missing File with "{unique_folder_suffix}" suffix in "{best_model_folder_path}"')
    elif len(model_folder) > 1:
        raise Exception(f'File with "{unique_folder_suffix}" suffix in "{best_model_folder_path}" is duplicated')
        
    # Get the prediction file
    folder = model_folder[0]
    best_model_path = os.path.join(best_model_folder_path, folder)
    
    # Get Hyperparameter
    best_model_hyperparameter = os.path.basename(best_model_path).split(unique_folder_suffix)[0]

    return best_model_name, best_model_hyperparameter, best_model_path 

In [26]:
best_model_name, best_model_hyperparameter, best_model_path = get_best_accuracy_model()
is_bert = best_model_name == "BERT"

print('Model Used')
print(f'  Type: {best_model_name}')
print(f'  Hyperparameter: {best_model_hyperparameter}')
print(f'  Path: {best_model_path}')

Model Used
  Type: BERT
  Hyperparameter: LR 2e-05, E 2, BS 16
  Path: ..\5) BERT Based Models\Model Results and Actual Data\BERT\Hyperparameter-1\LR 2e-05, E 2, BS 16 - Model


In [27]:
api = HfApi()
api.create_repo(repo_id=model_repo_name, exist_ok=True)
api.upload_folder(
    folder_path=best_model_path,
    repo_id=model_repo_name,
    repo_type="model",
    commit_message="Initial upload of sentiment model"
)

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

'https://huggingface.co/u-kuro/Thesis-2024-U.S.-Presidential-Election-Sentiment-Analysis/tree/main/'